In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
file = 'Resources/crime_data/NYPD_Arrests_Data__Historic_.csv'
df = pd.read_csv(file, encoding="ISO-8859-1")
df.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,190294601,11/23/2018,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200512,F,B,40,0.0,25-44,M,BLACK,1008096.0,233595.0,40.807816,-73.913863,POINT (-73.91386266099995 40.807816227000046)
1,190266280,11/21/2018,665.0,NaN,NaN,NaN,PL 4902001,F,Q,109,0.0,25-44,M,BLACK HISPANIC,1032084.0,216954.0,40.762044,-73.827330,POINT (-73.82732958099997 40.762043893000055)
2,190159521,11/19/2018,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200000,M,K,77,0.0,25-44,M,BLACK,1003508.0,185056.0,40.674600,-73.930575,POINT (-73.93057482199998 40.67459999000005)
3,190080571,11/16/2018,475.0,NaN,NaN,NaN,PL 1651601,M,Q,103,1.0,18-24,M,BLACK,1039726.0,195270.0,40.702482,-73.799922,POINT (-73.799922252 40.702482133000046)
4,190027735,11/15/2018,475.0,NaN,NaN,NaN,PL 1651601,M,M,10,1.0,45-64,M,WHITE HISPANIC,984669.0,210807.0,40.745301,-73.998489,POINT (-73.99848931699995 40.745301320000074)


In [3]:
df = df[[ "ARREST_DATE", "PD_DESC", "OFNS_DESC",  "ARREST_BORO", "ARREST_PRECINCT",  "AGE_GROUP", "PERP_SEX", "PERP_RACE"]].dropna()
df

,ARREST_DATE,PD_DESC,OFNS_DESC,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE
0,11/23/2018,"ASSAULT 2,1,UNCLASSIFIED",FELONY ASSAULT,B,40,25-44,M,BLACK
2,11/19/2018,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,K,77,25-44,M,BLACK
6,11/12/2018,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,M,33,45-64,F,BLACK
7,11/08/2018,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,K,69,<18,F,BLACK
8,11/06/2018,"TRESPASS 3, CRIMINAL",CRIMINAL TRESPASS,M,1,45-64,M,WHITE
...,...,...,...,...,...,...,...,...
5012951,01/11/2006,"TRAFFIC,UNCLASSIFIED INFRACTION",OTHER TRAFFIC INFRACTION,Q,104,18-24,M,WHITE HISPANIC
5012952,01/05/2006,"THEFT OF SERVICES, UNCLASSIFIED",OTHER OFFENSES RELATED TO THEFT,M,25,45-64,M,BLACK
5012953,01/05/2006,MANUFACTURE UNAUTHORIZED RECORDINGS,MISCELLANEOUS PENAL LAW,K,88,45-64,M,BLACK
5012954,01/10/2006,"TRAFFIC,UNCLASSIFIED INFRACTION",OTHER TRAFFIC INFRACTION,Q,108,25-44,M,WHITE


In [4]:
nypd_df = df.rename(columns={"ARREST_DATE": "arrest_date",
                        "PD_DESC": "police_description",
                        "OFNS_DESC": "offense_description",
                        "ARREST_BORO": "boro_code",
                        "ARREST_PRECINCT": "arresting_precinct",
                        "AGE_GROUP": "age_range",
                        "PERP_SEX": "sex",
                        "PERP_RACE": "race"
                                                })
nypd_df.head()

,arrest_date,police_description,offense_description,boro_code,arresting_precinct,age_range,sex,race
0,11/23/2018,"ASSAULT 2,1,UNCLASSIFIED",FELONY ASSAULT,B,40,25-44,M,BLACK
2,11/19/2018,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,K,77,25-44,M,BLACK
6,11/12/2018,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,M,33,45-64,F,BLACK
7,11/08/2018,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,K,69,<18,F,BLACK
8,11/06/2018,"TRESPASS 3, CRIMINAL",CRIMINAL TRESPASS,M,1,45-64,M,WHITE


In [5]:
rds_connection_string = "postgres:boozer11@localhost:5432/police-violence-racial-equity"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
engine.table_names()

['dallas', 'nypd']

In [7]:
nypd_df.to_sql(name = "nypd", con = engine, if_exists = "append", index = False)

In [8]:
pd.read_sql_query('select *  from nypd;',con=engine)

,arrest_date,police_description,offense_description,boro_code,arresting_precinct,age_range,sex,race
0,05/28/2017,"MISCHIEF,CRIMINAL, UNCL 2ND DEG 3RD DEG",CRIMINAL MISCHIEF & RELATED OFFENSES,Q,101,45-64,F,WHITE HISPANIC
1,06/04/2017,ASSAULT 3,ASSAULT 3 & RELATED OFFENSES,Q,105,25-44,M,BLACK
2,06/09/2017,"NY STATE LAWS,UNCLASSIFIED MISDEMEANOR",OTHER STATE LAWS (NON PENAL LAW),K,78,25-44,M,BLACK
3,05/27/2017,"LARCENY,GRAND FROM OPEN AREAS,UNCLASSIFIED",GRAND LARCENY,B,40,<18,F,WHITE HISPANIC
4,06/05/2017,"THEFT OF SERVICES, UNCLASSIFIED",OTHER OFFENSES RELATED TO THEFT,B,47,25-44,M,BLACK
...,...,...,...,...,...,...,...,...
5003901,06/05/2017,"LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED",PETIT LARCENY,K,77,18-24,M,BLACK
5003902,06/18/2017,OBSTR BREATH/CIRCUL,ASSAULT 3 & RELATED OFFENSES,Q,101,25-44,M,WHITE HISPANIC
5003903,05/31/2017,TAX LAW,OTHER STATE LAWS (NON PENAL LA,M,25,25-44,M,WHITE
5003904,06/08/2017,"TRESPASS 2, CRIMINAL",CRIMINAL TRESPASS,Q,109,25-44,M,BLACK
